In [2]:
!pip install wfdb
!pip install flask_cors
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 3.8 MB/s eta 0:00:0000:01


In [11]:
# # ============================================================================
# # FINAL INFERENCE & METRICS GENERATOR
# # ============================================================================
# # LOAD: esi_clean_best.pth (or any .pth you trained)
# # OUTPUT: AUC, Accuracy, Sensitivity, Specificity, F1
# # ============================================================================

# import os
# import sys
# import torch
# import torch.nn as nn
# import pandas as pd
# import numpy as np
# import wfdb
# import matplotlib.pyplot as plt
# import seaborn as sns
# from torch.utils.data import DataLoader, Dataset
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report
# from tqdm.auto import tqdm

# # --- 1. CONFIGURATION ---
# # Path to your BEST model checkpoint
# CHECKPOINT_PATH = "esi_finetune_clean/esi_clean_best.pth" 
# # Path to your CLEAN dataset
# CSV_PATH = "/kaggle/input/diab-csv/updated_heart_diabetes_subset.csv"
# DIABETIC_DIR = "/kaggle/input/mimic-iv-ecg-diabetic-subset/mimic-iv_diabetic_subset_extracted/mimic-iv_diabetic_subset_extracted"
# CONTROL_DIR = "/kaggle/input/mimic-iv-ecg-diabetic-subset/mimic-iv_non_diabetic_subset_extracted/mimic-iv_non_diabetic_subset_extracted"
# BATCH_SIZE = 64
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # --- 2. SETUP ESI ARCHITECTURE ---
# if not os.path.exists("ESI"):
#     os.system("git clone https://github.com/comp-well-org/ESI.git")
# sys.path.append(os.path.join(os.getcwd(), "ESI"))
# from model.convnextv2 import convnextv2_base

# # --- 3. RECREATE DATA SPLIT (CRITICAL) ---
# # We must use the exact same logic to get the correct Validation Set
# print("📊 Reconstructing Validation Set...")
# df = pd.read_csv(CSV_PATH)
# # Re-apply strict patient split logic used in training
# train_ids, val_ids = train_test_split(df['subject_id'].unique(), test_size=0.2, random_state=42)
# val_df = df[df['subject_id'].isin(val_ids)].copy()
# print(f"   Validation Records: {len(val_df)}")

# # --- 4. DATASET CLASS ---
# class MIMICInferenceDataset(Dataset):
#     def __init__(self, metadata, diabetic_dir, control_dir):
#         self.metadata = metadata
#         self.diabetic_dir = diabetic_dir
#         self.control_dir = control_dir

#     def __len__(self):
#         return len(self.metadata)

#     def __getitem__(self, idx):
#         row = self.metadata.iloc[idx]
#         base = self.diabetic_dir if row['label'] == 1 else self.control_dir
#         path = os.path.join(base, row['path'])
        
#         try:
#             data, _ = wfdb.rdsamp(path)
#             signal = data.T # [12, 5000]
#             mean = np.mean(signal, axis=1, keepdims=True)
#             std = np.std(signal, axis=1, keepdims=True)
#             signal = (signal - mean) / (std + 1e-6)
#             signal = np.nan_to_num(signal)
#             signal = np.clip(signal, -10, 10)
#             return torch.tensor(signal, dtype=torch.float32), torch.tensor(row['label'], dtype=torch.float32)
#         except:
#             return torch.zeros(12, 5000, dtype=torch.float32), torch.tensor(row['label'], dtype=torch.float32)

# val_loader = DataLoader(
#     MIMICInferenceDataset(val_df, DIABETIC_DIR, CONTROL_DIR), 
#     batch_size=BATCH_SIZE, shuffle=False, num_workers=4
# )

# # --- 5. LOAD MODEL ---
# class ESIInferenceModel(nn.Module):
#     def __init__(self, num_classes=1):
#         super().__init__()
#         # Initialize architecture
#         self.backbone = convnextv2_base(in_chans=12, num_classes=5, return_embedding=True)
#         self.classifier = nn.Linear(1024, num_classes)

#     def forward(self, x):
#         x = x.transpose(1, 2)
#         features = self.backbone(x)
#         return self.classifier(features)

# print(f"🔄 Loading Weights from: {CHECKPOINT_PATH}")
# model = ESIInferenceModel().to(DEVICE)

# # Handle DataParallel wrapping if it exists in checkpoint
# if torch.cuda.device_count() > 1:
#     model = nn.DataParallel(model)

# # Load state dict
# try:
#     state_dict = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
#     model.load_state_dict(state_dict)
#     print("✅ Weights Loaded Successfully.")
# except Exception as e:
#     print(f"❌ Error loading weights: {e}")
#     sys.exit(1)

# # --- 6. RUN INFERENCE ---
# print("🚀 Running Inference...")
# model.eval()
# all_preds = []
# all_labels = []

# with torch.no_grad():
#     for x, y in tqdm(val_loader):
#         x = x.to(DEVICE)
#         out = model(x)
#         preds = torch.sigmoid(out).cpu().numpy()
#         all_preds.extend(preds)
#         all_labels.extend(y.numpy())

# all_preds = np.array(all_preds).flatten()
# all_labels = np.array(all_labels).flatten()

# # --- 7. CALCULATE METRICS ---
# print("\n" + "="*40)
# print("       FINAL RESULTS REPORT       ")
# print("="*40)

# # 1. AUC
# auc = roc_auc_score(all_labels, all_preds)
# print(f"⭐ AUC: {auc:.4f}")

# # 2. Optimal Threshold (Youden's J statistic)
# fpr, tpr, thresholds = torch.tensor(all_labels), torch.tensor(all_preds), torch.tensor(all_preds)
# # Simple approach for threshold:
# # Find threshold that maximizes sensitivity + specificity - 1
# from sklearn.metrics import roc_curve
# fpr, tpr, thresholds = roc_curve(all_labels, all_preds)
# optimal_idx = np.argmax(tpr - fpr)
# optimal_threshold = thresholds[optimal_idx]
# print(f"⚖️  Optimal Threshold: {optimal_threshold:.4f}")

# # 3. Binary Metrics
# binary_preds = (all_preds > optimal_threshold).astype(int)
# acc = accuracy_score(all_labels, binary_preds)
# tn, fp, fn, tp = confusion_matrix(all_labels, binary_preds).ravel()

# sensitivity = tp / (tp + fn)
# specificity = tn / (tn + fp)
# ppv = tp / (tp + fp) # Precision
# npv = tn / (tn + fn)

# print(f"✅ Accuracy:    {acc:.4f}")
# print(f"❤️  Sensitivity: {sensitivity:.4f} (Recall)")
# print(f"🛡️  Specificity: {specificity:.4f}")
# print(f"🎯 Precision:   {ppv:.4f} (PPV)")
# print(f"📉 NPV:         {npv:.4f}")

# # --- 8. SAVE & PLOT ---
# # Save Predictions
# results_df = pd.DataFrame({
#     'label': all_labels,
#     'prediction_prob': all_preds,
#     'prediction_class': binary_preds
# })
# results_df.to_csv("final_predictions.csv", index=False)
# print("\n💾 Predictions saved to 'final_predictions.csv'")

# # Plot Confusion Matrix
# plt.figure(figsize=(6, 5))
# cm = confusion_matrix(all_labels, binary_preds)
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
#             xticklabels=['Control', 'Diabetic'],
#             yticklabels=['Control', 'Diabetic'])
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.title(f'Confusion Matrix (AUC: {auc:.3f})')
# plt.savefig('confusion_matrix.png')
# print("🖼️  Confusion Matrix saved to 'confusion_matrix.png'")

In [ ]:
# ============================================================================
# DIABETES DETECTION - KAGGLE FLASK INFERENCE CELL (FINAL VERSION)
# ALL PREPROCESSING MATCHES TRAINING EXACTLY
# Copy this entire cell into your Kaggle notebook and run AFTER training
# ============================================================================

import os
import sys

# Kill existing ngrok processes
os.system("killall ngrok")
os.system('pip install -q flask flask-cors pyngrok')

# Clone ESI if needed (assuming you already have it from training)
if not os.path.exists("ESI"):
    os.system("git clone https://github.com/comp-well-org/ESI.git")
if "ESI" not in sys.path:
    sys.path.append("ESI")

# ============================================================================
# IMPORTS
# ============================================================================
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok, conf
from kaggle_secrets import UserSecretsClient
import numpy as np
import torch
import torch.nn as nn
import wfdb
from werkzeug.utils import secure_filename
from datetime import datetime
import traceback
from model.convnextv2 import convnextv2_base

# ============================================================================
# 🔐 NGROK AUTHENTICATION
# ============================================================================
try:
    user_secrets = UserSecretsClient()
    NGROK_AUTH_TOKEN = user_secrets.get_secret("NGROK_AUTH_TOKEN_2")
    
    # OPTION 1: Use static domain (recommended)
    try:
        NGROK_STATIC_DOMAIN = user_secrets.get_secret("NGROK_STATIC_DOMAIN_2")
        USE_STATIC = True
        print(f"✅ Static domain loaded: {NGROK_STATIC_DOMAIN}")
    except:
        USE_STATIC = False
        print("⚠️  No static domain found, using random URL")
    
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    conf.get_default().region = "us"
    print("✅ Ngrok token loaded")
    
except Exception as e:
    print(f"❌ Secret retrieval failed: {e}")
    raise

# ============================================================================
# ⚙️ CONFIGURATION - UPDATE THESE VALUES
# ============================================================================

# 🔴 IMPORTANT: Update this path to your saved model
MODEL_PATH = "/kaggle/input/diab-csv/esi_clean_best.pth"
# Alternative if uploaded as dataset:
# MODEL_PATH = "/kaggle/input/your-diabetes-model/esi_clean_best.pth"

# 🔴 IMPORTANT: Update this with optimal threshold from training inference
# After running training inference cell, you'll see: "⚖️ Optimal Threshold: 0.XXXX"
# Copy that value here:
OPTIMAL_THRESHOLD = 0.4615  # <--- UPDATE THIS AFTER RUNNING TRAINING INFERENCE

# ============================================================================
# MODEL ARCHITECTURE (EXACT MATCH WITH TRAINING)
# ============================================================================
class ESIInferenceModel(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        self.backbone = convnextv2_base(in_chans=12, num_classes=5, return_embedding=True)
        self.classifier = nn.Linear(1024, num_classes)

    def forward(self, x):
        # Critical: transpose for ESI backbone
        x = x.transpose(1, 2)  # [B, 12, samples] -> [B, samples, 12]
        features = self.backbone(x)
        return self.classifier(features)

# ============================================================================
# LOAD TRAINED MODEL
# ============================================================================
print("\n" + "="*70)
print("🔄 LOADING DIABETES MODEL")
print("="*70)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

try:
    diabetes_model = ESIInferenceModel(num_classes=1).to(device)
    
    # Load checkpoint
    checkpoint = torch.load(MODEL_PATH, map_location=device)
    
    # Handle different checkpoint formats
    if isinstance(checkpoint, dict):
        if 'model_state_dict' in checkpoint:
            state_dict = checkpoint['model_state_dict']
        elif 'state_dict' in checkpoint:
            state_dict = checkpoint['state_dict']
        else:
            state_dict = checkpoint
    else:
        state_dict = checkpoint
    
    # Remove 'module.' prefix if saved with DataParallel
    new_state_dict = {}
    for k, v in state_dict.items():
        new_key = k[7:] if k.startswith("module.") else k
        new_state_dict[new_key] = v
    
    diabetes_model.load_state_dict(new_state_dict)
    diabetes_model.eval()
    print("✅ Diabetes model loaded successfully")
    print(f"✅ Using threshold: {OPTIMAL_THRESHOLD}")
    
except Exception as e:
    print(f"❌ Model loading failed: {e}")
    print(f"Attempted path: {MODEL_PATH}")
    traceback.print_exc()
    diabetes_model = None

print("="*70 + "\n")

# ============================================================================
# FLASK APP SETUP
# ============================================================================
app = Flask(__name__)
CORS(app)

UPLOAD_FOLDER = '/tmp/ecg_uploads_diabetes'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# ============================================================================
# PREPROCESSING FUNCTIONS (EXACT MATCH WITH TRAINING)
# ============================================================================

def extract_demographics(record_path):
    """Extract patient info from WFDB header"""
    try:
        header = wfdb.rdheader(record_path)
        demographics = {
            'record_name': header.record_name,
            'sampling_frequency': f"{header.fs} Hz",
            'signal_length': f"{header.sig_len} samples",
            'duration': f"{header.sig_len / header.fs:.1f} seconds",
            'number_of_signals': header.n_sig,
        }
        if hasattr(header, 'comments') and header.comments:
            for comment in header.comments:
                comment_lower = comment.lower()
                if 'age' in comment_lower: 
                    demographics['age'] = comment
                elif 'sex' in comment_lower or 'gender' in comment_lower: 
                    demographics['gender'] = comment
        return demographics
    except Exception as e: 
        return {'error': str(e)}

def generate_waveform_points(signal, target_points=800):
    """Generate SVG points for Lead I visualization"""
    try:
        downsample_factor = max(1, len(signal) // target_points)
        signal_viz = signal[::downsample_factor]
        signal_min, signal_max = signal_viz.min(), signal_viz.max()
        
        if abs(signal_max - signal_min) < 1e-6:
            signal_normalized = np.full_like(signal_viz, 50.0)
        else:
            signal_normalized = 100 - ((signal_viz - signal_min) / (signal_max - signal_min) * 80 + 10)
        
        x_scale = 800 / len(signal_normalized)
        points = [f"{i * x_scale:.2f},{y:.2f}" for i, y in enumerate(signal_normalized)]
        return " ".join(points)
    except: 
        return "0,50 800,50"

def preprocess_ecg_diabetes(record_path):
    """
    ⭐ EXACT MATCH WITH TRAINING PREPROCESSING ⭐
    
    Preprocess 12-lead ECG for diabetes detection
    - Uses natural signal length (no padding/cropping to fixed 5000)
    - Per-lead Z-score normalization
    - Clipping to [-10, 10]
    - NaN handling
    
    Returns: torch.Tensor [1, 12, samples]
    """
    try:
        # Load record - use whatever length it has (just like training)
        data, _ = wfdb.rdsamp(record_path)
        signal = data.T  # [leads, samples]
        
        print(f"   📊 Loaded signal: {signal.shape}")
        
        # Validate 12 leads
        if signal.shape[0] != 12:
            print(f"❌ Expected 12 leads, got {signal.shape[0]}")
            return None
        
        # ✅ DON'T force length - use natural length (EXACT MATCH with training)
        # Most MIMIC-IV ECGs are exactly 5000 samples naturally
        
        # ✅ Z-score normalization per lead (EXACT MATCH)
        mean = np.mean(signal, axis=1, keepdims=True)
        std = np.std(signal, axis=1, keepdims=True)
        signal = (signal - mean) / (std + 1e-6)
        
        # ✅ Clean (EXACT MATCH)
        signal = np.nan_to_num(signal)
        signal = np.clip(signal, -10, 10)
        
        print(f"   ✅ Preprocessed: {signal.shape}")
        
        # Return as tensor [1, 12, samples] - dynamic length
        return torch.tensor(signal, dtype=torch.float32).unsqueeze(0)
        
    except Exception as e:
        print(f"❌ Preprocessing failed: {e}")
        traceback.print_exc()
        return None

def run_diabetes_inference(model, signal_tensor, device):
    """
    Run diabetes inference with optimal threshold
    """
    try:
        model.eval()
        with torch.no_grad():
            output = model(signal_tensor.to(device))
            raw_logit = output.cpu().item()
            probability = torch.sigmoid(output).cpu().item()
        
        # Use optimal threshold from training
        has_diabetes = probability > OPTIMAL_THRESHOLD
        
        # Enhanced logging
        print(f"\n{'='*50}")
        print(f"🔍 DIABETES PREDICTION:")
        print(f"{'='*50}")
        print(f"Raw Logit:     {raw_logit:.4f}")
        print(f"Probability:   {probability:.4f}")
        print(f"Threshold:     {OPTIMAL_THRESHOLD}")
        print(f"Has Diabetes:  {has_diabetes}")
        print(f"{'='*50}\n")
        
        # Risk stratification (adjust these based on your validation results)
        if probability > 0.75:
            risk_level, risk_color = "High Risk", "danger"
        elif probability > 0.55:
            risk_level, risk_color = "Moderate Risk", "warning"
        else:
            risk_level, risk_color = "Low Risk", "success"
        
        return {
            'has_diabetes': bool(has_diabetes),
            'probability': float(probability),
            'raw_logit': float(raw_logit),
            'threshold_used': float(OPTIMAL_THRESHOLD),
            'risk_level': risk_level,
            'risk_color': risk_color,
            'diagnosis': 'Diabetes Risk Detected' if has_diabetes else 'No Diabetes Risk'
        }
        
    except Exception as e: 
        print(f"❌ Inference error: {e}")
        traceback.print_exc()
        return None

# ============================================================================
# API ENDPOINTS
# ============================================================================

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({
        'status': 'healthy',
        'model': 'diabetes',
        'model_loaded': diabetes_model is not None,
        'device': str(device),
        'gpu_available': torch.cuda.is_available(),
        'threshold': OPTIMAL_THRESHOLD
    })

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Validation
        if diabetes_model is None: 
            return jsonify({'success': False, 'error': 'Model not loaded'}), 500
        
        # Validate files
        if 'files' not in request.files: 
            return jsonify({'success': False, 'error': 'No files uploaded'}), 400
        
        files = request.files.getlist('files')
        hea_file = next((f for f in files if f.filename.endswith('.hea')), None)
        dat_file = next((f for f in files if f.filename.endswith('.dat')), None)
        
        if not hea_file or not dat_file: 
            return jsonify({'success': False, 'error': 'Need both .hea and .dat files'}), 400
        
        # Save uploaded files
        base_name = secure_filename(hea_file.filename).rsplit('.', 1)[0]
        hea_path = os.path.join(UPLOAD_FOLDER, f"{base_name}.hea")
        dat_path = os.path.join(UPLOAD_FOLDER, f"{base_name}.dat")
        hea_file.save(hea_path)
        dat_file.save(dat_path)
        
        print(f"\n📁 Processing: {base_name}")
        
        # Extract demographics
        record_path = os.path.join(UPLOAD_FOLDER, base_name)
        demographics = extract_demographics(record_path)
        
        # Load FULL signal for visualization (Lead I)
        full_signal, _ = wfdb.rdsamp(record_path)
        lead_i_signal = full_signal[:, 0] if full_signal.ndim > 1 else full_signal
        waveform_points = generate_waveform_points(lead_i_signal)
        
        # Preprocess for model (EXACT MATCH with training)
        signal_tensor = preprocess_ecg_diabetes(record_path)
        
        if signal_tensor is None:
            return jsonify({'success': False, 'error': 'Preprocessing failed - check if file has 12 leads'}), 500
        
        # Run inference
        prediction = run_diabetes_inference(diabetes_model, signal_tensor, device)
        
        if prediction is None:
            return jsonify({'success': False, 'error': 'Inference failed'}), 500
        
        # Build response
        response = {
            'success': True,
            'prediction': prediction,
            'demographics': demographics,
            'waveform': waveform_points,
            'timestamp': datetime.utcnow().isoformat()
        }
        
        # Cleanup
        try:
            os.remove(hea_path)
            os.remove(dat_path)
        except:
            pass
        
        print(f"✅ Analysis complete: {prediction['diagnosis']}")
        return jsonify(response), 200
        
    except Exception as e:
        print(f"\n❌ ERROR:")
        traceback.print_exc()
        return jsonify({'success': False, 'error': str(e)}), 500

# ============================================================================
# START SERVER
# ============================================================================

def start_diabetes_server():
    print("\n" + "="*70)
    print("🚀 STARTING DIABETES INFERENCE SERVER")
    print("="*70)
    
    if diabetes_model is None:
        print("❌ ERROR: Model not loaded!")
        return
    
    try:
        port = 5000
        
        # Connect with static domain or random URL
        if USE_STATIC:
            public_url = ngrok.connect(port, bind_tls=True, hostname=NGROK_STATIC_DOMAIN)
            url_display = f"https://{NGROK_STATIC_DOMAIN}"
        else:
            public_url = ngrok.connect(port)
            url_display = str(public_url)
        
        print("\n" + "="*70)
        print("🌐 DIABETES SERVER READY")
        print("="*70)
        print(f"🌍 Public URL: {url_display}")
        print(f"🔍 Health: {url_display}/health")
        print(f"📡 Predict: {url_display}/predict")
        print(f"⚖️  Using Threshold: {OPTIMAL_THRESHOLD}")
        print("="*70)
        print("\n⚠️  COPY THIS URL TO YOUR server.js:")
        print(f"    const DIABETES_URL = '{url_display}';")
        print("="*70 + "\n")
        
        app.run(host='0.0.0.0', port=port, debug=False, use_reloader=False)
        
    except Exception as e:
        print(f"\n❌ Server failed: {e}")
        traceback.print_exc()

# ============================================================================
# AUTO-START
# ============================================================================

if 'diabetes_model' in globals() and diabetes_model is not None:
    start_diabetes_server()
else:
    print("⚠️ Model not loaded. Check MODEL_PATH and re-run.")

# ============================================================================
# 📋 POST-TRAINING CHECKLIST
# ============================================================================
print("\n" + "="*70)
print("📋 CONFIGURATION CHECKLIST")
print("="*70)
print("1. ✅ Update MODEL_PATH (line 72)")
print(f"   Current: {MODEL_PATH}")
print("\n2. ⚠️  Update OPTIMAL_THRESHOLD (line 78)")
print(f"   Current: {OPTIMAL_THRESHOLD}")
print("   👉 Run training inference cell first to get optimal value")
print("\n3. ✅ Add Kaggle Secrets:")
print("   - NGROK_AUTH_TOKEN")
print("   - NGROK_DIABETES_DOMAIN (optional)")
print("="*70 + "\n")

ngrok: no process found


✅ Static domain loaded: seaworthy-superadequate-rebekah.ngrok-free.dev
✅ Ngrok token loaded

🔄 LOADING DIABETES MODEL
Device: cpu
✅ Diabetes model loaded successfully
✅ Using threshold: 0.4615


🚀 STARTING DIABETES INFERENCE SERVER

🌐 DIABETES SERVER READY
🌍 Public URL: https://seaworthy-superadequate-rebekah.ngrok-free.dev
🔍 Health: https://seaworthy-superadequate-rebekah.ngrok-free.dev/health
📡 Predict: https://seaworthy-superadequate-rebekah.ngrok-free.dev/predict
⚖️  Using Threshold: 0.4615

⚠️  COPY THIS URL TO YOUR server.js:
    const DIABETES_URL = 'https://seaworthy-superadequate-rebekah.ngrok-free.dev';

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.19.2.2:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.19.2.2:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit



📁 Processing: 46405379
   📊 Loaded signal: (12, 5000)
   ✅ Preprocessed: (12, 5000)


127.0.0.1 - - [14/Dec/2025 15:06:51] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Dec/2025 15:06:51] "POST /predict HTTP/1.1" 200 -


...


🔍 DIABETES PREDICTION:
Raw Logit:     -2.5768
Probability:   0.0706
Threshold:     0.4615
Has Diabetes:  False

✅ Analysis complete: No Diabetes Risk

📁 Processing: 48672900
   📊 Loaded signal: (12, 5000)
   ✅ Preprocessed: (12, 5000)


127.0.0.1 - - [14/Dec/2025 16:29:35] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Dec/2025 16:29:35] "POST /predict HTTP/1.1" 200 -



🔍 DIABETES PREDICTION:
Raw Logit:     -1.0749
Probability:   0.2545
Threshold:     0.4615
Has Diabetes:  False

✅ Analysis complete: No Diabetes Risk

📁 Processing: 41597721
   📊 Loaded signal: (12, 5000)
   ✅ Preprocessed: (12, 5000)


127.0.0.1 - - [14/Dec/2025 16:30:57] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Dec/2025 16:30:57] "POST /predict HTTP/1.1" 200 -



🔍 DIABETES PREDICTION:
Raw Logit:     0.8400
Probability:   0.6985
Threshold:     0.4615
Has Diabetes:  True

✅ Analysis complete: Diabetes Risk Detected
